In [1]:
import pandas as pd
from pathlib import Path

In [2]:
routes = pd.read_csv('routes.dat', sep='\t', usecols=['airline','A','B']).drop_duplicates()
airports = pd.read_csv('airport-codes.csv', sep=',')[['iata_code', 'iso_country', 'coordinates']]
airports = pd.concat([airports.drop(columns='coordinates'), airports.coordinates.str.split(', ', expand=True)], axis=1)
airports = airports[~airports.iata_code.isnull()].rename(columns={0:'lon',1:'lat', 'iata_code':'code', 'iso_country':'country'})

def save_df(airlines, name, coloring):
    df = routes[routes.airline.isin(airlines)]
    df = df.merge(airports, how='left', left_on='A', right_on='code').merge(airports, how='left', left_on='B', right_on='code', suffixes=('_A','_B'))
    df = df.drop(columns=['code_A','code_B'])
    df['color'] = df.apply(coloring, axis=1)
    df['AB'] = df.apply(lambda row: f"{row['A']}_{row['B']}", axis=1)

    df = df[(df.country_A=='US') & (df.country_B=='US')]

    df = df.sample(frac=1)
    
    p = Path(name)
    p.mkdir(exist_ok=True)
    df.to_csv(f'{name}/routes.csv', index=False)
    return df

# All

In [3]:
%%time
def coloring(row):
    if row.airline == 'AA': return '#00ffff'
    if row.airline == 'DL': return '#ff0000'
    if row.airline == 'UA': return '#426ae1'

save_df(['AA', 'DL', 'UA'], 'all', coloring)

CPU times: user 84.2 ms, sys: 4.84 ms, total: 89.1 ms
Wall time: 90 ms


,airline,A,B,country_A,lon_A,lat_A,country_B,lon_B,lat_B,color,AB
323,AA,DFW,HOU,US,-97.038002,32.896801,US,-95.27890015,29.64539909,#00ffff,DFW_HOU
1967,UA,DEN,SNA,US,-104.672996521,39.861698150635,US,-117.8679962,33.67570114,#426ae1,DEN_SNA
2312,UA,IAH,PBI,US,-95.34140014648438,29.984399795532227,US,-80.09559631347656,26.68320083618164,#426ae1,IAH_PBI
721,AA,ORD,FWA,US,-87.9048,41.9786,US,-85.19509888,40.97850037,#00ffff,ORD_FWA
536,AA,LGA,CAK,US,-73.872597,40.777199,US,-81.44219970703125,40.916099548339844,#00ffff,LGA_CAK
...,...,...,...,...,...,...,...,...,...,...,...
810,AA,PHL,BWI,US,-75.24109649658203,39.87189865112305,US,-76.668297,39.1754,#00ffff,PHL_BWI
2467,UA,ORD,FLL,US,-87.9048,41.9786,US,-80.152702,26.072599,#426ae1,ORD_FLL
1690,DL,RDU,PHL,US,-78.7874984741211,35.877601623535156,US,-75.24109649658203,39.87189865112305,#ff0000,RDU_PHL
488,AA,LAX,CMH,US,-118.407997,33.942501,US,-82.891899,39.998001,#00ffff,LAX_CMH


# American

In [4]:
%%time
def coloring(row):
    hubs = ['CLT','ORD','DFW','LAX','MIA','LGA','PHL', 'PHX', 'DCA']
    colors = ['#ff0000', '#ffff00', '#ffa500', '#00ffff', '#6cff7d', '#426ae1', '#fd68b3', '#426ae1', '#6cff7d']

    if row['A'] in hubs and row['B'] in hubs: return 'white'
    for k,v in zip(hubs, colors):
        if k in row.values: return v
    return 'gray'

save_df(['AA'], 'american', coloring)

CPU times: user 68.9 ms, sys: 2.69 ms, total: 71.6 ms
Wall time: 70.4 ms


,airline,A,B,country_A,lon_A,lat_A,country_B,lon_B,lat_B,color,AB
707,AA,ORD,CVG,US,-87.9048,41.9786,US,-84.667801,39.048801,#ffff00,ORD_CVG
43,AA,CLT,DAY,US,-80.94309997558594,35.2140007019043,US,-84.21939849853516,39.902400970458984,#ff0000,CLT_DAY
643,AA,MIA,PHX,US,-80.29060363769531,25.79319953918457,US,-112.01200103759766,33.43429946899414,white,MIA_PHX
907,AA,PHX,DSM,US,-112.01200103759766,33.43429946899414,US,-93.66310119628906,41.534000396728516,#426ae1,PHX_DSM
901,AA,PHX,BOI,US,-112.01200103759766,33.43429946899414,US,-116.223,43.5644,#426ae1,PHX_BOI
...,...,...,...,...,...,...,...,...,...,...,...
829,AA,PHL,GSO,US,-75.24109649658203,39.87189865112305,US,-79.93730163574219,36.097801208496094,#fd68b3,PHL_GSO
740,AA,ORD,MCO,US,-87.9048,41.9786,US,-81.30899810791016,28.429399490356445,#ffff00,ORD_MCO
235,AA,DEN,MIA,US,-104.672996521,39.861698150635,US,-80.29060363769531,25.79319953918457,#6cff7d,DEN_MIA
840,AA,PHL,LEX,US,-75.24109649658203,39.87189865112305,US,-84.60590362548828,38.0364990234375,#fd68b3,PHL_LEX


# Delta

In [5]:

def coloring(row):
    hubs = ['ATL','BOS','DTW','JFK','LAX','LGA','MSP','SEA','SLC']
    colors = ['#ff0000', '#ffff00', '#ffa500', '#00ffff', '#007f00', '#ffddad', '#fd68b3', '#426ae1', '#6cff7d']

    if row['A'] in hubs and row['B'] in hubs: return 'white'
    for k,v in zip(hubs, colors):
        if k in row.values: return v
    return 'gray'

save_df(['DL'], 'delta', coloring)

,airline,A,B,country_A,lon_A,lat_A,country_B,lon_B,lat_B,color,AB
76,DL,ATL,FWA,US,-84.428101,33.6367,US,-85.19509888,40.97850037,#ff0000,ATL_FWA
290,DL,CVG,STL,US,-84.667801,39.048801,US,-90.370003,38.748697,gray,CVG_STL
129,DL,ATL,MIA,US,-84.428101,33.6367,US,-80.29060363769531,25.79319953918457,#ff0000,ATL_MIA
368,DL,DTW,MQT,US,-83.35340118408203,42.212398529052734,US,-87.395401001,46.353599548300004,#ffa500,DTW_MQT
600,DL,MSP,ABR,US,-93.221802,44.882,US,-98.42179870605469,45.449100494384766,#fd68b3,MSP_ABR
...,...,...,...,...,...,...,...,...,...,...,...
671,DL,MSP,PHX,US,-93.221802,44.882,US,-112.01200103759766,33.43429946899414,#fd68b3,MSP_PHX
134,DL,ATL,MOB,US,-84.428101,33.6367,US,-88.242797851562,30.691200256348,#ff0000,ATL_MOB
786,DL,SLC,GJT,US,-111.97799682617188,40.78839874267578,US,-108.527000427,39.1223983765,#6cff7d,SLC_GJT
521,DL,LAX,MEM,US,-118.407997,33.942501,US,-89.97669982910156,35.04240036010742,#007f00,LAX_MEM


# United

In [6]:

def coloring(row):
    hubs = ['ORD','DEN','IAH','LAX','EWR','SFO','IAD']
    colors = ['#ff0000', '#ffff00', '#ffa500', '#00ffff', '#6cff7d', '#426ae1', '#fd68b3', '#426ae1', '#6cff7d']

    if row['A'] in hubs and row['B'] in hubs: return 'white'
    for k,v in zip(hubs, colors):
        if k in row.values: return v
    return 'gray'

save_df(['UA'], 'united', coloring)

,airline,A,B,country_A,lon_A,lat_A,country_B,lon_B,lat_B,color,AB
760,UA,SFO,ATL,US,-122.375,37.61899948120117,US,-84.428101,33.6367,#426ae1,SFO_ATL
371,UA,IAD,PHX,US,-77.45580292,38.94449997,US,-112.01200103759766,33.43429946899414,#fd68b3,IAD_PHX
648,UA,ORD,EAR,US,-87.9048,41.9786,US,-99.00679779,40.72700119,#ff0000,ORD_EAR
7,UA,CLE,MCO,US,-81.8498001099,41.4117012024,US,-81.30899810791016,28.429399490356445,gray,CLE_MCO
275,UA,EWR,SRQ,US,-74.168701171875,40.692501068115234,US,-82.55439758300781,27.39539909362793,#6cff7d,EWR_SRQ
...,...,...,...,...,...,...,...,...,...,...,...
520,UA,IAH,SAT,US,-95.34140014648438,29.984399795532227,US,-98.469803,29.533701,#ffa500,IAH_SAT
38,UA,DEN,COU,US,-104.672996521,39.861698150635,US,-92.219597,38.8181,#ffff00,DEN_COU
26,UA,DEN,BOI,US,-104.672996521,39.861698150635,US,-116.223,43.5644,#ffff00,DEN_BOI
706,UA,ORD,PBI,US,-87.9048,41.9786,US,-80.09559631347656,26.68320083618164,#ff0000,ORD_PBI
